<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задание-1." data-toc-modified-id="Задание-1.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задание 1.</a></span><ul class="toc-item"><li><span><a href="#Создание-таблицы-в-БД-PostgreSQL." data-toc-modified-id="Создание-таблицы-в-БД-PostgreSQL.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Создание таблицы в <code>БД</code> <code>PostgreSQL</code>.</a></span></li><li><span><a href="#Основной-запрос-с-использованием-CTE." data-toc-modified-id="Основной-запрос-с-использованием-CTE.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Основной запрос с использованием <code>CTE</code>.</a></span></li></ul></li><li><span><a href="#Задание-2." data-toc-modified-id="Задание-2.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Задание 2.</a></span><ul class="toc-item"><li><span><a href="#Создание-таблиц-в-БД-PostgreSQL." data-toc-modified-id="Создание-таблиц-в-БД-PostgreSQL.-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Создание таблиц в <code>БД</code> <code>PostgreSQL</code>.</a></span></li><li><span><a href="#Основной-запрос-с-использованием-CTE." data-toc-modified-id="Основной-запрос-с-использованием-CTE.-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Основной запрос с использованием <code>CTE</code>.</a></span></li></ul></li><li><span><a href="#Задание-3." data-toc-modified-id="Задание-3.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Задание 3.</a></span></li></ul></div>

# Тестовое задание на позицию `Аналитик данных (Junior)`.

In [1]:
# Импорт библиотек
import psycopg2 as ps
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Соединение с БД
conn = ps.connect(host="localhost",
                  port = 5432,
                  database="postgres",
                  user="postgres", password="123", options="-c search_path=public")

## Задание 1. 
Для согласования табеля за прошлый месяц, руководитель направления просит вас как можно скорее предоставить информацию по количеству дней за `январь 2025`, которые работники `Далекодорожный Ч.Ч.` и `Полуночный А.А.` провели в командировках. Сейчас они оба находятся на встречах и спросить у них самих нельзя, а ходить в бухгалтерию себе дороже, поэтому обратились к нам. `Господин Ч.Ч.` ездит в командировки на несколько дней, а `господин А.А.` обычно только на один. Командировки приходятся только на рабочие дни, если командировок нет, они работают в офисе, о чем есть данные из пропускной системы офиса.   
Составьте `sql-запрос`, используя (желательно) `PostgreSQL`, чтобы дать информацию руководителю. В данных из пропускной системы есть дата и время прохода всех сотрудников в офис. Данные хранятся в таблице `door_event` с полями: `Департамент (dept_id)`, `ай-ди устройства (skud_id)`, `ФИО (name)`, `дата и время события (dttm)`, `тип события (вход или выход) (event_type)`.


### Создание таблицы в `БД` `PostgreSQL`.
Данный скрипт нужно запускать в `БД` (например используя `DBeaver`).


In [3]:
df = pd.read_sql(''' 

SELECT *
FROM door_event
ORDER BY RANDOM()
LIMIT 10                            
''', con=conn)

df

,dept_id,skud_id,name,dttm,event_type
0,6,107,Полуночный А.А.,2024-12-02 09:15:00,вход
1,4,104,Кузнецов А.А.,2024-12-01 08:30:00,вход
2,6,107,Полуночный А.А.,2025-01-29 09:15:00,вход
3,5,105,Смирнова А.А.,2024-12-01 18:30:00,выход
4,6,107,Полуночный А.А.,2025-01-27 18:15:00,выход
5,2,102,Петров П.П.,2025-01-21 08:15:00,вход
6,6,107,Полуночный А.А.,2024-12-11 18:15:00,выход
7,1,101,Иванов И.И.,2025-01-21 17:00:00,выход
8,1,101,Иванов И.И.,2024-12-01 08:00:00,вход
9,5,105,Смирнова А.А.,2025-01-23 09:30:00,вход


### Основной запрос с использованием `CTE`.

In [4]:
df = pd.read_sql(''' 
-- Создаем временные таблицы:
WITH 
-- С днями января 2025 года
working_days AS (
SELECT generate_series('2025-01-09'::date, 
                       '2025-01-31'::date, 
                       '1 day'::interval
                       )::date AS work_date
),

-- С рабочими днями января 2025 года с записями для необходимых сотрудников из БД
workers_days AS (
SELECT name, 
       DATE(dttm) AS date_w, 
       count(event_type) AS event_cnt
FROM door_event
WHERE EXTRACT(DOW FROM dttm) BETWEEN 1 AND 5
  AND name IN ('Далекодорожный Ч.Ч.', 'Полуночный А.А.') 
  AND DATE(dttm) BETWEEN '2025-01-09' AND '2025-01-31'
GROUP BY name, DATE(dttm)
),

-- С всеми рабочими днями января 2025 года для каждого из сотрудников
cross_table AS (
SELECT work_date,
      'Далекодорожный Ч.Ч.' AS worker
FROM working_days
WHERE EXTRACT(DOW FROM work_date) BETWEEN 1 AND 5
UNION ALL 
SELECT work_date,
      'Полуночный А.А.' AS worker
FROM working_days
WHERE EXTRACT(DOW FROM work_date) BETWEEN 1 AND 5
)

-- Итоговый запрос
SELECT worker,
       COUNT(work_date) AS business_trips
FROM cross_table
LEFT JOIN workers_days ON cross_table.worker = workers_days.name AND cross_table.work_date =  workers_days.date_w
WHERE name IS NULL
GROUP BY worker
''', con=conn)

df

,worker,business_trips
0,Далекодорожный Ч.Ч.,15
1,Полуночный А.А.,12


## Задание 2. 
Коллеги из бухгалтерии обратились к Вам за помощью, пока администратор 1С отдыхает в Апатитах. Они просят выгрузить из базы данных сотрудников, кто потратил с корпоративных банковских карт больше `20 000 рублей` за `2024 год` и имевших до совершения первой топливной транзакции `(product_nm = ‘Fuel’)` в `2024 году` более `50 000 рублей` на счету. Требуется вывести `ФИО сотрудника`, `кол-во транзакций списания на топливо` и `средний чек списания за 2024 год`.      

***Структура таблиц следующая:***   

Таблица с сотрудниками:   
```
Employee_id	Employee_name

Empl_id_1	Empl_name_1
Empl_id_2	Empl_name_2
Empl_id_3	Empl_name_3
…	…
```

Таблица с продуктами:   
```
Product id	Product_nm

Product id 1	Fuel
Product id 2	Tickets
Product id 3	Hotel 
…	…
```
Таблица с транзакциями, выгруженная из 1С и приведенная к 3НФ:    
```
Empl id	    Txn ID	    Txn_date	Txn_type	TXN_sts	    Product id	    Amount	Balance_after txn

Empl id_1	Txn ID 1	01.01.2022	Accrual	    Cancelled	Product id 1	Amount1	Balance_after txn 1
Empl id_2	Txn ID 2	27.03.2022	Redemption	Processed	Product id 2	Amount2	Balance_after txn 2
Empl id_3	Txn ID 3	12.11.2022	Redemption	Cancelled	Product id 3	Amount3	Balance_after txn 3
…	…	…	…	…	…	…	…
```

`Txn_date` – дата транзакции,   
`Txn_type` – тип транзакции (`redemption` – списание, `accrual` – начисление (перевод денег подотчет)),   
`Txn_sts` – статус транзакции (`Processed` – обработанная, `cancelled` – отменённая),   
`Product_id` – идентификатор продукта,   
`Amount` – деньги, начисленные / списанные за данную транзакцию,   
`Product_nm` – название продукта (`Fuel` – топливо, `Tickets` – билеты на самолет или поезд, `Hotel` – деньги на оплату отеля).       

На все продукты деньги могут начисляться и списываться: `начисление` – это перевод из бухгалтерии подотчет, `списание` – покупка товаров или услуг определенной категории. 


### Создание таблиц в `БД` `PostgreSQL`.
Данный скрипт нужно запускать в `БД` (например используя `DBeaver`).


In [5]:
data = pd.read_sql(''' 
SELECT *
FROM employees
''', con=conn)

data

,employee_id,employee_name
0,Empl id_1,Empl_name_1
1,Empl id_2,Empl_name_2
2,Empl id_3,Empl_name_3
3,Empl id_4,Empl_name_4
4,Empl id_5,Empl_name_5
...,...,...
95,Empl id_96,Empl_name_96
96,Empl id_97,Empl_name_97
97,Empl id_98,Empl_name_98
98,Empl id_99,Empl_name_99


In [6]:
data = pd.read_sql(''' 
SELECT *
FROM products
''', con=conn)

data

,product_id,product_nm
0,1,Fuel
1,2,Tickets
2,3,Hotel
3,4,Fuel
4,5,Tickets
...,...,...
95,96,Hotel
96,97,Fuel
97,98,Tickets
98,99,Hotel


In [7]:
data = pd.read_sql(''' 
SELECT *
FROM transactions
''', con=conn)

data

,empl_id,txn_id,txn_date,txn_type,txn_sts,product_id,amount,balance_after_txn
0,Empl id_1,Txn ID 1,2022-01-01,Accrual,Cancelled,1,5000.0,10000.0
1,Empl id_2,Txn ID 2,2022-03-27,Redemption,Processed,2,20000.0,50000.0
2,Empl id_3,Txn ID 3,2022-11-12,Redemption,Cancelled,3,10000.0,100000.0
3,Empl id_4,Txn ID 4,2024-01-09,Accrual,Cancelled,75,1934.0,30557.0
4,Empl id_5,Txn ID 5,2024-01-11,Redemption,Processed,48,6374.0,11507.0
...,...,...,...,...,...,...,...,...
1647,Empl id_96,Txn ID 96,2026-08-18,Accrual,Cancelled,65,138.0,60075.0
1648,Empl id_97,Txn ID 97,2026-08-28,Redemption,Processed,57,90.0,93152.0
1649,Empl id_98,Txn ID 98,2026-09-07,Accrual,Cancelled,79,2423.0,27377.0
1650,Empl id_99,Txn ID 99,2026-09-17,Redemption,Processed,72,4196.0,66710.0


### Основной запрос с использованием `CTE`.

In [8]:
data = pd.read_sql(''' 
-- Создаем временные таблицы:
WITH 
-- Находим все топливные транзакции за 2024 год
fuel_transactions AS (
SELECT t.Empl_id,
       t.Txn_date,
       t.Amount,
       t.Balance_after_txn
FROM transactions AS t
INNER JOIN products AS p ON t.Product_id = p.Product_id
WHERE p.Product_nm = 'Fuel' 
  AND t.Txn_type = 'Redemption' 
  AND t.Txn_sts = 'Processed' 
  AND EXTRACT(YEAR FROM t.Txn_date) = 2024
),

-- Считаем общую сумму списаний за 2024 год для каждого сотрудника
employee_total_spent AS (
SELECT t.Empl_id,
       SUM(t.Amount) AS total_spent
FROM transactions t
WHERE t.Txn_type = 'Redemption'
  AND t.Txn_sts = 'Processed' 
  AND EXTRACT(YEAR FROM t.Txn_date) = 2024
GROUP BY t.Empl_id
HAVING SUM(t.Amount) > 20000
),

-- Находим первую топливную транзакцию для каждого сотрудника в 2024 году
first_fuel_transaction AS (
SELECT ft.Empl_id,
       MIN(ft.Txn_date) AS first_fuel_date
FROM fuel_transactions ft
GROUP BY ft.Empl_id
),

-- Находим дату транзакции до первой топливной транзакции
date_before_first_fuel AS (
SELECT fft.Empl_id,
       MAX(t.Txn_date) AS date_balance_before_fuel
FROM first_fuel_transaction AS fft
JOIN transactions AS t ON fft.Empl_id = t.Empl_id
WHERE t.Txn_date < fft.first_fuel_date 
  AND t.Txn_sts = 'Processed'
GROUP BY fft.Empl_id
),

-- Находим баланс до первой топливной транзакции более 50000 рублей
balance_before_first_fuel AS (
SELECT fft.Empl_id,
       t.Balance_after_txn AS balance_before_fuel
FROM date_before_first_fuel AS fft
INNER JOIN transactions AS t ON fft.Empl_id = t.Empl_id 
  AND fft.date_balance_before_fuel = t.Txn_date
WHERE t.Balance_after_txn > 50000
)

-- Итоговый запрос
SELECT Employee_name AS ФИО, 
       COUNT(Txn_date) AS Количество_топливных_транзакций, 
       AVG(ft.Amount)::NUMERIC(10, 2) AS Средний_чек
FROM employee_total_spent AS  ets
INNER JOIN balance_before_first_fuel AS bbff USING(Empl_id)
LEFT JOIN employees AS e ON ets.Empl_id = e.Employee_id
LEFT JOIN fuel_transactions AS ft ON ets.Empl_id = ft.Empl_id
GROUP BY e.Employee_name;
''', con=conn)

data

,ФИО,Количество_топливных_транзакций,Средний_чек
0,Empl_name_11,5,5503.00
1,Empl_name_19,7,4324.00
2,Empl_name_23,5,6560.20
3,Empl_name_27,2,4377.00
4,Empl_name_31,4,3690.50
5,Empl_name_49,2,8159.50
6,Empl_name_7,7,4255.29
7,Empl_name_75,2,4135.50
8,Empl_name_77,1,8661.00
9,Empl_name_83,1,2401.00


## Задание 3. 

В одном холдинге работают два деревообрабатывающих предприятия: одно находится в `Стерлитамаке`, другое – в `Нарьян-Маре`.   
Они имеют одинаковый штат сотрудников и основные производственные фонды, сырье закупают у ближайших поставщиков. Изготавливая одинаковое количество продукции на продажу, одна фирма имеет `на 25% больше` прибыли, чем другая. Опишите факторы, на которые стоит обратить внимание аналитику, чтобы подтянуть отстающую фирму к показателям лидирующей, если это возможно. 


**Для анализа ситуации и выявления причин разницы в прибыли между двумя предприятиями, аналитику следует обратить внимание на следующие факторы:**

1. **Себестоимость продукции.**      
Стоимость сырья: Убедитесь, что сырье закупается по схожим ценам. Возможно, поставщики в одном регионе предлагают менее выгодные условия.    
Энергоресурсы: Стоимость электроэнергии, газа и других ресурсов может отличаться в зависимости от региона.

2. **Производительность труда.**     
Эффективность сотрудников: Проверьте, есть ли различия в производительности труда между двумя предприятиями. Возможно, в одном из них выше текучесть кадров или ниже квалификация сотрудников.    
Оборудование: Убедитесь, что оборудование на обоих предприятиях одинаково современно и эффективно. Возможно, на одном из них оборудование чаще ломается или требует большего обслуживания.    
Производственные процессы: Оцените технологические процессы на каждом из предприятий. Возможно, одно из них использует более современные и эффективные технологии или оборудование, что позволяет снижать затраты или увеличивать производительность.    
Уровень автоматизации: Сравните уровень автоматизации и механизации процессов. Более автоматизированное производство может быть более эффективным.    

3. **Логистика и сбыт.**    
Расходы на доставку готовой продукции: Если одно предприятие находится ближе к основным рынкам сбыта, это может снижать транспортные расходы.    
Цены на продукцию: Проверьте, одинаковы ли цены на продукцию на обоих предприятиях. Возможно, одно из них продает продукцию по более низким ценам из-за конкуренции или других факторов.   

4. **Управленческие расходы.**    
Административные затраты: Проверьте, нет ли различий в управленческих расходах. Возможно, одно из предприятий тратит больше на административный персонал или офисные расходы.    
Эффективность управления: Оцените, насколько эффективно используются ресурсы на каждом предприятии. Возможно, в одном из них есть избыточные запасы или неэффективное планирование.    

5. **Маркетинг и продажи.**    
Реклама и продвижение: Проверьте, одинаковы ли затраты на маркетинг и продвижение продукции. Возможно, одно из предприятий тратит больше на рекламу, что увеличивает расходы.      
Рынок и конкуренция: Проанализируйте рыночную ситуацию и конкуренцию в регионах. Возможно, одно предприятие имеет более сильные позиции на рынке.

6. **Качество продукции.**    
Cтандарты качества: Изучите качество продукции, выпускаемой каждым предприятием. Высокое качество может способствовать повышению цен на продукцию и увеличению спроса.    
Возвраты и рекламации: Анализируйте количество возвратов и рекламаций на продукцию. Высокий уровень дефектов может негативно сказываться на прибыли.

7. **Финансовые факторы.**    
Кредитная нагрузка: Если одно из предприятий имеет более высокую кредитную нагрузку, это может увеличивать финансовые расходы.  
Управление запасами: Оцените, насколько эффективно управляются запасы сырья и готовой продукции.

**Итоговые рекомендации:**    
 - Проведите детальный анализ себестоимости продукции на обоих предприятиях.   
 - Сравните производительность труда и эффективность использования оборудования.   
 - Оцените логистические и сбытовые расходы.   
 - Сравните цены на продукцию.   
 - Убедитесь, что управленческие расходы и маркетинговая стратегия одинаково эффективны.     

**После анализа этих факторов можно разработать конкретные меры для повышения прибыли отстающего предприятия.**   

Например:   
 - Оптимизировать закупки сырья.   
 - Улучшить логистику.   
 - Повысить квалификацию сотрудников.   
 - Внедрить более эффективные технологии производства.   